# Web scraping Toronto neighborhood data

### Importing required libraries

In [2]:
import requests
import pandas as pd
import csv

### Installing BeautifulSoup if not already installed

In [2]:
#!conda install beautifulsoup4
from bs4 import BeautifulSoup

### Web scraping to obtain required neighbourhood data from wikipedia

In [36]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')
file = open('postal_codes_toronto.csv', 'w')
csv_wr = csv.writer(file)
csv_wr.writerow(['PostalCode', 'Borough', 'Neighbourhood'])

table = soup.find('table', class_ = 'wikitable')
table_rows = table.find_all('tr')
postal_codes = []
boroughs = [] 
neighbourhoods = [] 

for row in table_rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  
            
            postalcode = columns[0].text.split('\n')[0]
            postal_codes.append(postalcode)
            borough = columns[1].text.split('\n')[0]
            boroughs.append(borough)
            neighbourhood = columns[2].text.split('\n')[0]    
            neighbourhoods.append(neighbourhood)
            
    except Exception as e : 
        pass    
    
    
postal_present = []
for index_i, postal_i in enumerate(postal_codes) :   
    if postal_i not in postal_present :
        nbds = neighbourhoods[index_i]
        for index_f, postal_f in enumerate(postal_codes) :
            if postal_i == postal_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_wr.writerow([postal_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postal_present.append(postal_i)
file.close()

### Creating a dataframe

In [96]:
df = pd.read_csv('postal_codes_toronto.csv')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [98]:
df.shape

(180, 3)

In [99]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Drpping NaN values

In [100]:
df2 = df.dropna()
df2.reset_index(drop=True, inplace=True)

In [101]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [102]:
df2.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Replacing / in neighbourhood with ,

In [103]:
df2['Neighbourhood'] = df2.Neighbourhood.str.replace('/',',')

C:\Users\kt5702\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [113]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [108]:
df2 = df2.astype(str)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


### storing the final cleaned data into a csv file

In [109]:
df2.to_csv('Toronto_neighbourhoods.csv')

In [115]:
df2.shape

(103, 3)

### Obtaining latitude and longitude information from the csv file

In [25]:
df_loc = pd.read_csv("Geospatial_Coordinates.csv")

In [26]:
df_loc.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [27]:
df_loc.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [24]:
df_nbd = pd.read_csv('Toronto_neighbourhoods.csv',index_col=[0])
df_nbd.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [21]:
df_loc.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [28]:
df_final = pd.merge(df_nbd,df_loc,how="inner")

In [29]:
df_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [30]:
df_final.to_csv('Latitude_longitude_data.csv')